In [2]:
import pandas as pd
import warnings
import locale

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.3f}'.format
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [11]:
%%sql
SELECT
    divtec_notasfiscais.data,
    divtec_notasfiscais.produto,
    round(sum(divtec_notasfiscais.volume),2) as volume_produto_dia
FROM
    divtec_notasfiscais
WHERE data >='2024-08-01' and data <='2024-08-31'
GROUP BY
    divtec_notasfiscais.produto,
    divtec_notasfiscais.data,
    divtec_notasfiscais.volume

,data,produto,volume_produto_dia
0,2024-08-01,00101-ABACATE,9.000
1,2024-08-01,00101-ABACATE,54.000
2,2024-08-01,00101-ABACATE,72.000
3,2024-08-01,00101-ABACATE,54.000
4,2024-08-01,00101-ABACATE,80.000
...,...,...,...
18844,2024-08-23,ZAMIOCULCA,8.000
18845,2024-08-26,ZAMIOCULCA,8.000
18846,2024-08-26,ZAMIOCULCA,36.000
18847,2024-08-27,ZAMIOCULCA,8.000


In [49]:
dados_2024 = pd.read_excel('Janeiro 2024.xlsx')
dados_2024 = dados_2024[['data', 'produto', 'VOLUME']]
dados_2024.rename(columns={'VOLUME':'volume_produto_dia'}, inplace=True)
dados_2024['data'] = dados_2024['data'].astype('datetime64[ns]')

In [50]:
dados_2024.head()

,data,produto,volume_produto_dia
0,2024-01-02,00101-ABACATE,36.000
1,2024-01-02,00101-ABACATE,90.000
2,2024-01-02,00101-ABACATE,100.000
3,2024-01-02,00101-ABACATE,108.000
4,2024-01-02,00101-ABACATE,180.000


In [114]:
%%sql
CALL CotacaoEmQuilogramasPorPeriodo('2024-01-01','2024-02-01');

,data,produto,unidade,peso em Kg,mais_frequente,Preço por Kg
0,2024-01-04,00101-ABACATE,KG,1.000,5.000,5.000
1,2024-01-04,00201-ABACAXI CAIENA (HAWAI),UND,1.800,9.000,5.000
2,2024-01-04,00202-ABACAXI PEROLA,UND,1.300,8.000,6.150
3,2024-01-04,00203-ABACAXI TERRA DE AREIA,UND,0.400,4.500,11.250
4,2024-01-04,00301-AMEIXA NACIONAL,KG,1.000,8.000,8.000
...,...,...,...,...,...,...
1557,2024-01-30,54601-ROSA VERMELHA 20UN,MOL,0.566,40.000,70.670
1558,2024-01-30,54701-MOSQUITINHO,MOL,0.300,29.000,96.670
1559,2024-01-30,54801-VERDES (SAMAMBAIA PRETA),MOL,0.500,15.000,30.000
1560,2024-01-30,54901-ROSA COLORIDA 20UN,MOL,0.600,40.000,66.670


In [115]:
cotacao[['data','produto','Preço por Kg']].to_excel('cotação Jan 2024.xlsx', index=False)

In [98]:
cotacao['data'] = cotacao['data'].astype('datetime64[ns]') 

In [99]:
cotacao = cotacao.groupby(['produto','unidade'])['mais_frequente'].mean().reset_index()
cotacao.rename(columns={'mais_frequente':'Preço por Kg'},inplace=True)

In [100]:
# notas = notas.copy(deep=True)
notas = dados_2024
notas = notas.groupby(['produto'])['volume_produto_dia'].sum().reset_index()

In [106]:
volume = notas.merge(cotacao, on=['produto'])

In [107]:
volume = volume[['produto','volume_produto_dia','Preço por Kg']]

In [108]:
volume['total'] = volume['volume_produto_dia'] * volume['Preço por Kg']
produtos = volume.copy(deep=True)

In [109]:
volume = volume[['produto','volume_produto_dia','total']]
produtos = produtos[['produto','volume_produto_dia','total']]

In [110]:
volume.loc[volume['total'].isnull()]

,produto,volume_produto_dia,total


In [111]:
volume.rename(columns={'volume_produto_dia':'volume','total':'valor comercializado'},inplace=True)
produtos.rename(columns={'volume_produto_dia':'volume','total':'valor comercializado'},inplace=True)

In [113]:
volume[['volume','valor comercializado']].sum()

volume                  46614800.505
valor comercializado   421941936.924
dtype: float64

In [61]:
volume_valor_por_dia = volume.groupby(['data'])[['volume','valor comercializado']].sum()
volume_valor_por_dia

,volume,valor comercializado
data,,
2024-01-04,2940109.186,15945225.560
2024-01-09,2448727.128,13009814.451
2024-01-11,2868816.665,14324314.331
2024-01-16,2241549.356,10801518.294
2024-01-18,4948436.888,24098182.630
2024-01-23,2683752.584,12256027.571
2024-01-25,2421898.067,12044143.985
2024-01-30,2341573.995,11651459.079


In [42]:
volume_valor_por_dia.to_excel('volume e valor Janeiro 2024.xlsx')

In [ ]:
produtos = produtos.groupby(['produto'])[['volume','valor comercializado']].sum().reset_index()
produtos

In [ ]:
produtos = produtos.sort_values(by=['valor comercializado'],ascending=False)
produtos.to_excel('produtos 09-09 até 13-09.xlsx')